# 真实数据测试

基于baseline，我们在本notebook中参考项目给出的demo，尝试采用真实的数据运行项目。

## 环境准备

tqdm是Python中使用图形界面展示迭代过程的模块。

In [1]:
# import tqdm, tqdm.notebook
# tqdm.tqdm = tqdm.notebook.tqdm  # 为了实现进度条的图形效果

# from time import sleep

# # eg.
# for i in tqdm.tqdm(range(10)):
#     sleep(0.1)

numpy用于处理数据，pathlib用于记录文件目录及文件操作。

In [2]:
import numpy as np
from pathlib import Path
import math
import os

引入hloc包，各个模块的作用如下：

- extract_features: 提取图像特征，将特征写入文件
- pairs_from_exhaustive: 构建用于特征匹配的图像对
- match_features: 在图像对间做特征匹配
- reconstruction: 依据特征、图像对和特征匹配的结果构建三维模型

In [3]:
from hloc import extract_features, pairs_from_exhaustive, match_features, reconstruction
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

from hloc import visualization

Python中使用h5py模块处理hdf5文件。

In [4]:
import h5py

## 构建

指定需要使用的输出路径。

In [5]:
images = Path('datasets/0904_left_5869')
outputs = Path('outputs/0904_left_5869')

# 每次输出前，将输出文件夹删除
!rm -rf $outputs

sfm_pairs = outputs / 'pairs-sfm-blocks'  # 建图阶段存储用于特征匹配的图像对
loc_pairs = outputs / 'pairs-loc.txt'  # 定位阶段存储用于特征匹配的图像对
sfm_dir = outputs / 'sfm-blocks'    # sectional sfm model
features = outputs / 'features-blocks'  # 特征输出文件，以hdf5的格式存储
matches = outputs / 'matches-blocks'

## 3D建图

展示用于建图的图像数据。

In [6]:

# 迭代获取制定目录下的图像名称
references = [p.relative_to(images).as_posix() for p in (images / 'mapping/').iterdir()]
references.sort()
for idx, ref in enumerate(references):
    end = ' ' if (idx+1) % 5 else '\n'

    print(ref, end=end)

# 输出图像数目
lens = len(references)
print(lens, "mapping images")

# 绘制图像
# plot_images([read_image(images / r) for r in references], dpi=25)

num_per_chunk = 200
total_chunks = math.ceil(lens / num_per_chunk)
print(total_chunks)
chunks = []
for i in range(total_chunks):
    start = i*num_per_chunk if i ==0 else math.ceil(i*num_per_chunk-0.1*num_per_chunk)
    end = min(i*num_per_chunk + num_per_chunk, lens)
    chunks.append(references[start: end])

print(len(chunks), len(chunks[0]), len(chunks[1]), len(chunks[len(chunks)-1]))


mapping/1693816943750013256.png mapping/1693816943800013392.png mapping/1693816943850013535.png mapping/1693816943950013798.png mapping/1693816944000013933.png
mapping/1693816944100014211.png mapping/1693816944150014339.png mapping/1693816944200014466.png mapping/1693816944250014609.png mapping/1693816944350014792.png
mapping/1693816944400014856.png mapping/1693816944450014937.png mapping/1693816944550015097.png mapping/1693816944600015170.png mapping/1693816944650015250.png
mapping/1693816944750015411.png mapping/1693816944800015491.png mapping/1693816944850015572.png mapping/1693816944950015724.png mapping/1693816945000015813.png
mapping/1693816945050015893.png mapping/1693816945150016046.png mapping/1693816945200016126.png mapping/1693816945300016024.png mapping/1693816945350015859.png
mapping/1693816945400015718.png mapping/1693816945500015404.png mapping/1693816945550015256.png mapping/1693816945600015115.png mapping/1693816945700014801.png
mapping/1693816945750014660.png mapping/

### 特征提取

特征提取得到hdf5文件，其中每张图像包含以下信息：

- image_size：图像尺寸
- keypoints：关键点，每个关键点对应一个分数和一个描述符
- descriptors: 关键点的描述符
- keypoint_scores：关键点分数，通常越高代表当前点越关键

在feature_conf中，可以指定特征提取采取的模型。其中，如disk模型等，可能限制了最大关键点数目。

In [7]:
# 配置选用的模型
feature_conf = extract_features.confs['disk']
for block_idx, block_img in enumerate(chunks):
    if not os.path.exists(features):
        os.makedirs(features)
    # if not os.path.exists(matches):
    #     os.makedirs(matches)
    features_output_path = features / '{}.h5'.format('features_block_' + str(block_idx))

    print(features_output_path)
    extract_features.main(feature_conf, images, image_list=block_img, feature_path=features_output_path)


[2023/09/05 19:43:16 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}


outputs/0904_left_5869/features-blocks/features_block_0.h5


100%|██████████| 200/200 [01:09<00:00,  2.90it/s]
[2023/09/05 19:44:26 hloc INFO] Finished exporting features.
[2023/09/05 19:44:26 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}


outputs/0904_left_5869/features-blocks/features_block_1.h5


100%|██████████| 220/220 [01:15<00:00,  2.92it/s]
[2023/09/05 19:45:41 hloc INFO] Finished exporting features.
[2023/09/05 19:45:41 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}


...

100%|██████████| 220/220 [01:09<00:00,  3.17it/s]
[2023/09/05 20:15:19 hloc INFO] Finished exporting features.
[2023/09/05 20:15:19 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}


outputs/0904_left_5869/features-blocks/features_block_27.h5


100%|██████████| 220/220 [01:09<00:00,  3.17it/s]
[2023/09/05 20:16:28 hloc INFO] Finished exporting features.
[2023/09/05 20:16:28 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}


outputs/0904_left_5869/features-blocks/features_block_28.h5


100%|██████████| 150/150 [00:47<00:00,  3.16it/s]
[2023/09/05 20:17:16 hloc INFO] Finished exporting features.


In [8]:
# 打印特征(.h5)文件中每张图片存储的信息
# with h5py.File(str(features), 'r', libver='latest') as fd:
#     for attr in fd['mapping']['1399381453828679.png']:
#         print(attr)

In [9]:
# fig_name = '1399381453828679.png'

# with h5py.File(str(features), 'r', libver='latest') as fd:
#     print(f"The size of {fig_name} is ({fd['mapping'][fig_name]['image_size'][0]}, {fd['mapping'][fig_name]['image_size'][1]})", end='\n\n')

#     print(f"There are {len(fd['mapping'][fig_name]['keypoints'])} keypoints in {fig_name}", end=', ')
#     print('each keypoint has a score of itself.')
#     print(f"eg. The first key_point in {fig_name} is {fd['mapping'][fig_name]['keypoints'][0]}", end=', ')
#     print(f"with score {fd['mapping'][fig_name]['keypoint_scores'][0]}", end='\n\n')

#     print(f"The shape of descriptors is {fd['mapping'][fig_name]['descriptors'].shape}")
    

### 创建图像对

图像对的含义是用于特征匹配的图像对，hloc包中提供了多种的构建图像对的方式：

- `pair_from_exhaustive`，为提供的图像或特征列表两两间构建一个pair

In [10]:
for block_idx, block_img in enumerate(chunks):
    if not os.path.exists(sfm_pairs):
        os.makedirs(sfm_pairs)
    pairs_output_path = sfm_pairs / '{}.txt'.format('sfmpairs_block_' + str(block_idx))

    print(pairs_output_path)
    pairs_from_exhaustive.main(pairs_output_path, image_list=block_img)

[2023/09/05 20:17:16 hloc INFO] Found 19900 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.


[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO] Found 24090 pairs.
[2023/09/05 20:17:16 hloc INFO]

outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_0.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_1.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_2.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_3.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_4.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_5.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_6.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_7.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_8.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_9.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_10.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_11.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_12.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_13.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_14.txt
outputs/0904_left_5869/pairs-sfm-blocks/sfmpairs_block_15.txt
outputs/0904_left_

### 特征匹配

> **Structure from Motion**(SfM)是基于收集到的无序图片进行三维重建的离线算法。

对于之前得到的特征和待匹配的图像对，其中的每对都需要进行特征匹配。从一张图片的一个关键点出发，寻找另一张图片中与这个关键点距离最小的点，除此之外，还需要满足一定的条件，如在SIFT中要求

$$
\frac{与最近点之间的距离}{与次近点之间的距离} \le Threshold
$$

在matcher_conf中可以选择特征匹配采用的模型。

In [11]:
matcher_conf = match_features.confs['disk+lightglue']
for block_idx, block_img in enumerate(chunks):
    if not os.path.exists(matches):
        os.makedirs(matches)
    matches_output_path = matches / '{}.h5'.format('matches_block_' + str(block_idx))
    features_input_path = features / '{}.h5'.format('features_block_' + str(block_idx))
    sfmpairs_input_path = sfm_pairs / '{}.txt'.format('sfmpairs_block_' + str(block_idx))
    # features_input_path = os.path.join(features, f'features_block_{block_idx}.h5')
    # sfmpairs_input_path = os.path.join(sfm_pairs, f'sfmpairs_block_{block_idx}.txt')
    print(matches_output_path)
    print(features_input_path)
    match_features.main(matcher_conf, pairs=sfmpairs_input_path, features=features_input_path, matches=matches_output_path)

[2023/09/05 20:17:16 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}


outputs/0904_left_5869/matches-blocks/matches_block_0.h5
outputs/0904_left_5869/features-blocks/features_block_0.h5


100%|██████████| 19900/19900 [22:50<00:00, 14.52it/s]
[2023/09/05 20:40:07 hloc INFO] Finished exporting matches.
[2023/09/05 20:40:07 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}


outputs/0904_left_5869/matches-blocks/matches_block_1.h5
outputs/0904_left_5869/features-blocks/features_block_1.h5


100%|██████████| 24090/24090 [29:29<00:00, 13.62it/s]
[2023/09/05 21:09:36 hloc INFO] Finished exporting matches.
[2023/09/05 21:09:36 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}


...

100%|██████████| 24090/24090 [32:09<00:00, 12.48it/s]
[2023/09/06 10:01:53 hloc INFO] Finished exporting matches.
[2023/09/06 10:01:53 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}


outputs/0904_left_5869/matches-blocks/matches_block_27.h5
outputs/0904_left_5869/features-blocks/features_block_27.h5


100%|██████████| 24090/24090 [26:55<00:00, 14.91it/s]
[2023/09/06 10:28:49 hloc INFO] Finished exporting matches.
[2023/09/06 10:28:49 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}


outputs/0904_left_5869/matches-blocks/matches_block_28.h5
outputs/0904_left_5869/features-blocks/features_block_28.h5


100%|██████████| 11175/11175 [10:16<00:00, 18.12it/s]
[2023/09/06 10:39:06 hloc INFO] Finished exporting matches.


### 3D模型重建

在得到了图像特征、图像对和匹配结果后，可以构建出3D模型。大致思路是，借助那些匹配的图像对来估计图片的视角（相机位姿、点云以及场景的结构），利用多个图像对之间产生的多个视角得到模型。

In [12]:
import os

if not os.path.exists(sfm_dir):
        os.makedirs(sfm_dir)

models = {}
for block_idx, block_img in enumerate(chunks):
    model_output_path = sfm_dir / '{}'.format('model_block_' + str(block_idx))
    matches_input_path = matches / '{}.h5'.format('matches_block_' + str(block_idx))
    features_input_path = features / '{}.h5'.format('features_block_' + str(block_idx))
    sfmpairs_input_path = sfm_pairs / '{}.txt'.format('sfmpairs_block_' + str(block_idx))

    models[block_idx] = reconstruction.main(model_output_path, images, sfmpairs_input_path, features_input_path, matches_input_path, image_list=block_img)

[2023/09/06 10:39:06 hloc INFO] Creating an empty database...
[2023/09/06 10:39:07 hloc INFO] Importing images into the database...
[2023/09/06 10:39:12 hloc INFO] Importing features into the database...
100%|██████████| 200/200 [00:00<00:00, 1954.15it/s]
[2023/09/06 10:39:12 hloc INFO] Importing matches into the database...
100%|██████████| 19900/19900 [00:11<00:00, 1671.97it/s]
[2023/09/06 10:39:25 hloc INFO] Performing geometric verification of the matches...
[2023/09/06 10:39:41 hloc INFO] Running 3D reconstruction...
W0906 10:58:45.287119 67539 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0906 10:58:45.299142 67539 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0906 10:58:45.302274 67539 levenberg_marquardt_strategy.cc:123] Linear solver failure. Failed to compute a step: Eige

In [ ]:
import open3d as o3d
pcd = []
for block_idx, block_img in enumerate(chunks):
    pcd[block_idx] = o3d.io.read_point_cloud(sfm_dir/"points3D.bin")

fused_pcd = o3d.geometry.PointCloud.concatenate(pcd[0], pcd[1])

ModuleNotFoundError: No module named 'open3d'

展示构建出的模型。

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

将结果进行可视化，其中的点表示在该图像上提取到的特征，红色表示该特征点未用于3D模型的构建，蓝色的点则表示用于了模型构建。

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='visibility', n=2)

## 定位

在构建好3D模型后，可以输入图片进行定位，估计相机位姿。

### 准备数据

选择与建图图像拍摄自同一时刻，但视角不同的图像。

In [ ]:
# query = 'query/16.jpg'  # 制定查询图像
# url = "https://upload.wikimedia.org/wikipedia/commons/5/53/Paris_-_Basilique_du_Sacr%C3%A9_Coeur%2C_Montmartre_-_panoramio.jpg"
# try other queries by uncommenting their url
# url = "https://upload.wikimedia.org/wikipedia/commons/5/59/Basilique_du_Sacr%C3%A9-C%C5%93ur_%285430392880%29.jpg"
# url = "https://upload.wikimedia.org/wikipedia/commons/8/8e/Sacr%C3%A9_C%C5%93ur_at_night%21_%285865355326%29.jpg"
query = 'query/night.jpg'
# !mkdir -p $images/query && wget $url -O $images/$query -q

plot_images([read_image(images / query)], dpi=75)  # 展示图像

### 特征提取与匹配

特征提取和匹配与之前并无差异，但构建图像对时，与之前自匹配不同，现在是以用于建图的图像作为参考(ref)，而以定位图像作为待匹配图像。

In [ ]:
# 提取图像中的特征
extract_features.main(feature_conf, images, image_list=[query], feature_path=features, overwrite=True)

# 生成用于特征匹配的图像对
pairs_from_exhaustive.main(loc_pairs, image_list=[query], ref_list=references)

# 特征匹配
match_features.main(matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True)

### 估计相机位姿

首先我们读取图像的文件头，初步得到相机的参数（焦距等）。

In [ ]:
import pycolmap

camera = pycolmap.infer_camera_from_image(images / query)

camera.focal_length  # 输出焦距

使用PnP+RANSAC估计相机绝对位姿，细化相机参数。

In [ ]:
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster

# 生成每张图片的列表
ref_ids = [model.find_image_with_name(r).image_id for r in references]


conf = {
    # 估计采用RANSAC算法
    'estimation': {'ransac': {'max_error': 12}},
    # 细化相机焦距，额外参数
    'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
}

# 依据配置生成定位器
localizer = QueryLocalizer(model, conf)

# 位姿估计
ret, log = pose_from_cluster(localizer, query, camera, ref_ids, features, matches)

print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')

# 对于上面的结果作可视化
visualization.visualize_loc_from_log(images, query, log, model)

上述代码核心在于返回了ret，包含我们需要用到的位姿信息:

- qvec：表示旋转的四元数
- tvec：相机的位置向量
- num_inliers：模型中认为得到了正确匹配的点，用来计算相机位姿

In [ ]:
ret['qvec'], ret['tvec'], ret['num_inliers']

依据得到的位姿，在3D模型中画出相机。

In [ ]:
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=query, fill=True)
fig.show()

可视化inliers，即那些认为正确匹配的点。

In [ ]:
# 取出正确匹配的点的索引
inl_3d = np.array([model.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])

# 绘制
viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query)

fig.show()